In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import os

import numpy as np

from spikeforest2_utils import AutoRecordingExtractor, AutoSortingExtractor
import kachery as ka

Autosaving every 180 seconds


In [ ]:
# SET ROOT DIRECTORY WHERE DATA WILL BE SAVED
root_dir = '/media/cat/4TBSSD/data/spikingforest/hybrid_janelia/static_siprobe/rec_16c_1200s_11/'


In [3]:
#LOAD RAW DATA

# Configure kachery to download data from the public database
ka.set_config(fr='default_readonly')

# Specify the recording and ground truth sorting
# The addresses files can be obtained from this link: https://spikeforest.flatironinstitute.org/recordings
# see datasets on the left hand side of this website
# Ask Cat (or Flatiron people) about which datasets to focus on 
recording_path = 'sha1dir://615aa23efde8898aa89002613e20ad59dcde42f9.hybrid_janelia/static_siprobe/rec_16c_1200s_11'

# LOAD DATA
print ("...loading recording...")
recording = AutoRecordingExtractor(recording_path, download=True)
data = recording.get_traces()
data = data.T # need to transpose from C order to Fortran order for YASS; final shape data = [n_times, n_chans]
print ("DATA:", data.shape)
data.tofile(os.path.join(root_dir,'data.bin')  # 

print ("DONE LOADING / SAVING RAW DATA")

...loading recording...
Downloaded file (28.4 KiB) in 1.2543187141418457 sec.
Downloaded file (1.1 GiB) in 263.47198128700256 sec.
DONE


In [6]:
# LOAD GROUND TRUTH SPIKETIMES
print ("...loading ground truth data...")
sorting_true_path = os.path.join(recording_path,'firings_true.mda')
sorting_true = AutoSortingExtractor(sorting_true_path)

ids = sorting_true.get_unit_ids()
print (" neuron ids: ", ids)
spike_train = np.zeros((0,2),'int32')
for id_ in ids:
    temp = sorting_true.get_unit_spike_train(id_)
    temp2 = np.zeros((temp.shape[0],2),'int32')
    temp2[:,0] = temp
    temp2[:,1] = id_
    spike_train = np.vstack((spike_train, temp2))
spike_train[:,1]-=1
print (" spike train: ", spike_train)
ground_truth_dir = os.path.join(root_dir,'ground_truth')

# make ground-truth directory
try:
    os.mkdir(ground_truth_dir)
except:
    pass

np.save(ground_truth_dir+'/spike_train_ground_truth.npy', spike_train)

print ("DONE SAVING GROUND TRUTH SPIKETIMES")

Downloaded file (11.1 MiB) in 2.559777021408081 sec.
DONE


In [13]:
# Visualize data
data = data.T # need to transpose from C order to Fortran order for YASS
print ("DATA (Should be [n_times, n_chans]):", data.shape)
#data.tofile(os.path.join(root_dir,'data.bin')  # might need to ravel, but not always required.

for k in range(data.shape[1]):    
    plt.plot(data[:10000,k]+k*100,c='black')
plt.show()
